# Feature calculation



Molpipeline provides multiple molecular featurization methods and descriptors from RDKit. This notebook shows how features like

- Morgan binary fingerprints
- Morgan count fingerprints
- MACCS keys fingerprints
- Physicochemical features

can be easily calculated in parallel and in different variations with MolPipeline. If you are interested in further molecular featurization and descriptors check out the `molpipeline.mol2any` module.

In [1]:
import pandas as pd

from molpipeline import Pipeline
from molpipeline.any2mol import AutoToMol
from molpipeline.mol2any import MolToMACCSFP, MolToMorganFP, MolToRDKitPhysChem

In this example we fetch the ESOL (delaney) data set. However, you can use any other data set.

In [2]:
df_full = pd.read_csv(
    "https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/delaney-processed.csv",
    usecols=lambda col: col != "num",
)

We use a smaller portion of the data set for illustration

In [3]:
df = df_full.head(n=100)
df

,Compound ID,ESOL predicted log solubility in mols per litre,Minimum Degree,Molecular Weight,Number of H-Bond Donors,Number of Rings,Number of Rotatable Bonds,Polar Surface Area,measured log solubility in mols per litre,smiles
0,Amigdalin,-0.974,1,457.432,7,3,7,202.32,-0.77,OCC3OC(OCC2OC(OC(C#N)c1ccccc1)C(O)C(O)C2O)C(O)...
1,Fenfuram,-2.885,1,201.225,1,2,2,42.24,-3.30,Cc1occc1C(=O)Nc2ccccc2
2,citral,-2.579,1,152.237,0,0,4,17.07,-2.06,CC(C)=CCCC(C)=CC(=O)
3,Picene,-6.618,2,278.354,0,5,0,0.00,-7.87,c1ccc2c(c1)ccc3c2ccc4c5ccccc5ccc43
4,Thiophene,-2.232,2,84.143,0,1,0,0.00,-1.33,c1ccsc1
...,...,...,...,...,...,...,...,...,...,...
95,diethylstilbestrol,-5.074,1,268.356,2,2,4,40.46,-4.07,CCC(=C(CC)c1ccc(O)cc1)c2ccc(O)cc2
96,Chlorothalonil,-3.995,1,265.914,0,1,0,47.58,-5.64,c1(C#N)c(Cl)c(C#N)c(Cl)c(Cl)c(Cl)1
97,"2,3',4',5-PCB",-6.312,1,291.992,0,2,1,0.00,-7.25,Clc1ccc(Cl)c(c1)c2ccc(Cl)c(Cl)c2
98,styrene oxide,-1.826,2,120.151,0,2,1,12.53,-1.60,C1OC1c2ccccc2


## Calculating fingerprints

### Morgan binary fingerprints

Morgan fingerprints are the most popular molecular fingerprints. They are also known as [Extended-Connectivity Fingerprints (ECFP)](https://doi.org/10.1021/ci100050t). They encode circular substructures in the molecule. The binary version contains only 0s and 1s indicating the presence or absence of the substructures in the molecule.

Let's define the Pipeline to first read the molecule and then calculate the binary Morgan fingerprint. Then, we execute it by calling the `transform` function.

In [4]:
%%time
# define the pipeline
pipeline_morgan = Pipeline(
    [("auto2mol", AutoToMol()), ("morgan2_2048", MolToMorganFP(n_bits=2048, radius=2))],
    n_jobs=-1,
)
# execute the pipeline
morgan_matrix = pipeline_morgan.transform(df["smiles"])
morgan_matrix

CPU times: user 181 ms, sys: 247 ms, total: 428 ms
Wall time: 12.6 s


<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 2191 stored elements and shape (100, 2048)>

By default, the `MolToMorganFP` element returns a sparse matrix. More specifically, a [csr_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html) is returned which is more memory efficient than a dense matrix since most elements in the matrix are zero.

To get a dense matrix you can convert the `csr_matrix` to a dense numpy matrix like this:

In [5]:
morgan_matrix.todense()

matrix([[0, 1, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0]])

Alternatively, you can specify in the `MolToMorganFP` element the return type of the feature matrix by using the `return_as` option. You can choose between

- `return_as="sparse"` which returns a `csr_matrix`
- `return_as="dense` which returns a dense numpy matrix
- `return_as="explicit_bit_vect"` which returns RDKit's dense [ExplicitBitVect](https://www.rdkit.org/new_docs/cppapi/classExplicitBitVect.html)

In [6]:
%%time
pipeline_morgan_dense = Pipeline(
    [
        ("auto2mol", AutoToMol()),
        ("morgan2_2048", MolToMorganFP(n_bits=2048, radius=2, return_as="dense")),
    ],
    n_jobs=-1,
)
dense_morgan_matrix = pipeline_morgan_dense.transform(df["smiles"])
dense_morgan_matrix

CPU times: user 45.4 ms, sys: 11.7 ms, total: 57 ms
Wall time: 62.4 ms


array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]], dtype=uint8)

The feature matrix can be used to train a machine learning model but also for various analyses.

### Morgan count fingerprints

Just set `counted=True` to compute Morgan count fingerprints instead of binary fingerprints.

In [7]:
pipeline_morgan_counted = Pipeline(
    [
        ("auto2mol", AutoToMol()),
        (
            "morgan2_2048",
            MolToMorganFP(n_bits=2048, radius=2, counted=True, return_as="dense"),
        ),
    ],
    n_jobs=-1,
)
count_morgan_matrix = pipeline_morgan_counted.transform(df["smiles"])
count_morgan_matrix

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]], dtype=uint32)

When we sort the matrix values we see that some substructures are present up to 14 times in a single molecule.

In [8]:
sorted(count_morgan_matrix.ravel(), reverse=True)[:20]

[14, 13, 12, 12, 11, 10, 10, 10, 10, 10, 10, 10, 9, 9, 8, 8, 8, 8, 8, 8]

### MACCS key fingerprints

MACCS keys are a manually defined set of 166 substructures whose presence is checked in the molecule. MACCS keys contain for example common functional groups.

In [9]:
%%time
pipeline_maccs_dense = Pipeline(
    [("auto2mol", AutoToMol()), ("maccs", MolToMACCSFP(return_as="dense"))],
    n_jobs=-1,
)
dense_maccs_matrix = pipeline_maccs_dense.transform(df["smiles"])
dense_maccs_matrix

CPU times: user 43.8 ms, sys: 1.15 ms, total: 44.9 ms
Wall time: 70.9 ms


array([[0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 0, 1, 0]])

## Physicochemical features

RDKit also provides more than 200 physicochemical descriptors that can readily be computed from most molecules. In MolPipeline we can compute these features with the `MolToRDKitPhysChem` element.

In [10]:
%%time
pipeline_physchem = Pipeline(
    [("auto2mol", AutoToMol()), ("physchem", MolToRDKitPhysChem(standardizer=None))],
    n_jobs=-1,
)
physchem_matrix = pipeline_physchem.transform(df["smiles"])
physchem_matrix

CPU times: user 68.1 ms, sys: 2.43 ms, total: 70.5 ms
Wall time: 171 ms


array([[10.25332888, 10.25332888,  0.48660209, ...,  0.        ,
         0.        ,  0.        ],
       [11.72491119, 11.72491119,  0.14587963, ...,  0.        ,
         0.        ,  0.        ],
       [10.02049761, 10.02049761,  0.84508976, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 6.08815823,  6.08815823,  0.49556374, ...,  0.        ,
         0.        ,  0.        ],
       [ 5.09453704,  5.09453704,  0.40851852, ...,  0.        ,
         0.        ,  0.        ],
       [ 2.2037037 ,  2.2037037 ,  0.65851852, ...,  0.        ,
         0.        ,  0.        ]])

We can get the name of the descriptors like this:

In [11]:
pipeline_physchem["physchem"].descriptor_list[:20]

['MaxAbsEStateIndex',
 'MaxEStateIndex',
 'MinAbsEStateIndex',
 'MinEStateIndex',
 'qed',
 'SPS',
 'HeavyAtomMolWt',
 'ExactMolWt',
 'NumValenceElectrons',
 'NumRadicalElectrons',
 'MaxPartialCharge',
 'MinPartialCharge',
 'MaxAbsPartialCharge',
 'MinAbsPartialCharge',
 'FpDensityMorgan1',
 'FpDensityMorgan2',
 'FpDensityMorgan3',
 'BCUT2D_MWHI',
 'BCUT2D_MWLOW',
 'BCUT2D_CHGHI']

When we only want to calculate a subset of all available descriptors we can specify this during pipeline construction

In [12]:
%%time
pipeline_physchem_small = Pipeline(
    [
        ("auto2mol", AutoToMol()),
        (
            "physchem",
            MolToRDKitPhysChem(
                standardizer=None,
                descriptor_list=["HeavyAtomMolWt", "TPSA", "NumHAcceptors"],
            ),
        ),
    ],
    n_jobs=-1,
)
physchem_matrix_small = pipeline_physchem_small.transform(df["smiles"])
physchem_matrix_small

CPU times: user 41.2 ms, sys: 3.38 ms, total: 44.6 ms
Wall time: 47.5 ms


array([[430.216, 202.32 ,  12.   ],
       [190.137,  42.24 ,   2.   ],
       [136.109,  17.07 ,   1.   ],
       [264.242,   0.   ,   0.   ],
       [ 80.111,   0.   ,   1.   ],
       [130.151,  12.89 ,   2.   ],
       [321.397,   0.   ,   0.   ],
       [248.196,  40.46 ,   2.   ],
       [372.849,  12.53 ,   1.   ],
       [372.247,  63.22 ,   6.   ],
       [ 78.05 ,  29.1  ,   1.   ],
       [155.563,   0.   ,   0.   ],
       [ 60.055,   0.   ,   0.   ],
       [204.144,  58.2  ,   2.   ],
       [168.154,   0.   ,   0.   ],
       [ 71.486,   0.   ,   0.   ],
       [ 76.054,  20.23 ,   1.   ],
       [ 98.084,  23.79 ,   1.   ],
       [283.184,  53.47 ,   6.   ],
       [148.12 ,  20.23 ,   1.   ],
       [321.397,   0.   ,   0.   ],
       [216.155,  54.86 ,   3.   ],
       [243.25 ,  18.46 ,   5.   ],
       [166.115,  38.33 ,   2.   ],
       [309.139, 115.54 ,   6.   ],
       [100.076,  20.23 ,   1.   ],
       [172.103,  72.68 ,   5.   ],
       [196.121,  75.27 ,   